# Data preprocessing

In [1]:
pwd

'C:\\Users\\quay\\Documents\\ShareIt\\notebooks'

In [2]:
cd ..\data\raw

C:\Users\quay\Documents\ShareIt\data\raw


## Imports

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random
import secrets
import string
from pathlib import Path
import re

## Reading files

In [4]:
users = pd.read_csv("amazon_prime_users.csv")

In [5]:
items = pd.read_csv("ElectronicsData.csv")

## Data cleaning

### *users Schema*

In [6]:
users.head()

,User ID,Name,Email Address,Username,Date of Birth,Gender,Location,Membership Start Date,Membership End Date,Subscription Plan,Payment Information,Renewal Status,Usage Frequency,Purchase History,Favorite Genres,Devices Used,Engagement Metrics,Feedback/Ratings,Customer Support Interactions
0,1,Ronald Murphy,williamholland@example.com,williamholland,1953-06-03,Male,Rebeccachester,2024-01-15,2025-01-14,Annual,Mastercard,Manual,Regular,Electronics,Documentary,Smart TV,Medium,3.6,3
1,2,Scott Allen,scott22@example.org,scott22,1978-07-08,Male,Mcphersonview,2024-01-07,2025-01-06,Monthly,Visa,Manual,Regular,Electronics,Horror,Smartphone,Medium,3.8,7
2,3,Jonathan Parrish,brooke16@example.org,brooke16,1994-12-06,Female,Youngfort,2024-04-13,2025-04-13,Monthly,Mastercard,Manual,Regular,Books,Comedy,Smart TV,Low,3.3,8
3,4,Megan Williams,elizabeth31@example.net,elizabeth31,1964-12-22,Female,Feliciashire,2024-01-24,2025-01-23,Monthly,Amex,Auto-renew,Regular,Electronics,Documentary,Smart TV,High,3.3,7
4,5,Kathryn Brown,pattersonalexandra@example.org,pattersonalexandra,1961-06-04,Male,Port Deborah,2024-02-14,2025-02-13,Annual,Visa,Auto-renew,Frequent,Clothing,Drama,Smart TV,Low,4.3,1


In [7]:
users.isna().sum().sum()

0

In [8]:
users.shape

(2500, 19)

#### Change columns to PascalCase 

In [9]:
users.columns = users.columns.str.replace(' ', '')

In [10]:
users.columns

Index(['UserID', 'Name', 'EmailAddress', 'Username', 'DateofBirth', 'Gender',
       'Location', 'MembershipStartDate', 'MembershipEndDate',
       'SubscriptionPlan', 'PaymentInformation', 'RenewalStatus',
       'UsageFrequency', 'PurchaseHistory', 'FavoriteGenres', 'DevicesUsed',
       'EngagementMetrics', 'Feedback/Ratings', 'CustomerSupportInteractions'],
      dtype='object')

In [11]:
users[['FirstName', 'LastName']] = users['Name'].str.split(' ', n = 1, expand=True)

In [12]:
users2 = users[["FirstName", "LastName", "EmailAddress"]].copy()

In [13]:
users2 = users2.rename(columns={'EmailAddress': 'Email'})

In [14]:
users2.head()

,FirstName,LastName,Email
0,Ronald,Murphy,williamholland@example.com
1,Scott,Allen,scott22@example.org
2,Jonathan,Parrish,brooke16@example.org
3,Megan,Williams,elizabeth31@example.net
4,Kathryn,Brown,pattersonalexandra@example.org


In [15]:
users2.isna().sum().sum()

0

In [16]:
def generate_unique_phone_numbers(count):
    phone_numbers = set()
    while len(phone_numbers) < count:
        start_digit = np.random.choice([8, 9])
        
        remaining_digits = np.random.randint(0, 10, 7)
        phone_number = str(start_digit) + ''.join(map(str, remaining_digits))
        phone_numbers.add(phone_number)
    return list(phone_numbers)

num_records = len(users)
unique_phone_numbers = generate_unique_phone_numbers(num_records)

In [17]:
users2['Contact'] = unique_phone_numbers

In [18]:
users2.head()

,FirstName,LastName,Email,Contact
0,Ronald,Murphy,williamholland@example.com,96358820
1,Scott,Allen,scott22@example.org,94481144
2,Jonathan,Parrish,brooke16@example.org,90340963
3,Megan,Williams,elizabeth31@example.net,89049086
4,Kathryn,Brown,pattersonalexandra@example.org,88751848


In [19]:
users2['Contact'].nunique()

2500

In [20]:
def generate_password(length=10):
    characters = string.ascii_letters + string.digits
    return ''.join(random.choice(characters) for i in range(length))



In [21]:
users2['Password'] = users.apply(lambda x: generate_password(), axis=1)

In [22]:
groupby = users2.groupby('Email').size().reset_index(name='Count')
groupby[groupby['Count'] > 1].head()

,Email,Count
319,bryandiaz@example.com,2
540,dbailey@example.net,2
577,djohnson@example.org,2
681,ewilson@example.com,2
956,jacqueline84@example.com,2


In [23]:
users2 = users2.drop_duplicates(subset=['Email'], keep='first')

In [24]:
groupby = users2.groupby('Email').size().reset_index(name='Count')
groupby[groupby['Count'] > 1].head()

,Email,Count


In [25]:
users2['UserID'] = range(1, len(users2) + 1)

In [26]:
users2 = users2[["UserID", "FirstName", "LastName", "Email", "Password", "Contact"]]

In [27]:
users2.head()

,UserID,FirstName,LastName,Email,Password,Contact
0,1,Ronald,Murphy,williamholland@example.com,xJJFwREZpJ,96358820
1,2,Scott,Allen,scott22@example.org,vdSj8pYGtj,94481144
2,3,Jonathan,Parrish,brooke16@example.org,zX0cm9xmcX,90340963
3,4,Megan,Williams,elizabeth31@example.net,VcKDj0b4Op,89049086
4,5,Kathryn,Brown,pattersonalexandra@example.org,RUCAQXHRHZ,88751848


In [28]:
users2.isna().sum().sum()

0

In [29]:
users2.shape

(2480, 6)

### Unit test

In [30]:
def is_valid_email(email: str) -> bool:
    email_pattern = r'^[a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+$'
    return bool(re.match(email_pattern, email))

def test_email_format(df):
    invalid_emails = df[~users2['Email'].apply(is_valid_email)]
    assert invalid_emails.empty, f"Invalid emails found: {invalid_emails['Email'].tolist()}"
    print("all emails valid")
test_email_format(users2)

all emails valid


In [31]:
def test_duplicate_emails(df):
    duplicate_emails = df[df.duplicated(subset=['Email'], keep=False)]
    assert duplicate_emails.empty, f"Duplicate emails found: {duplicate_emails['Email'].tolist()}"
    print("No duplicate emails found.")

test_duplicate_emails(users2)


No duplicate emails found.


### *items Schema*

In [32]:
items.head()

,Sub Category,Price,Discount,Rating,Title,Currency,Feature
0,Batteries,$20.99,No Discount,NaN,"Duracell Coppertop Alkaline AA Batteries, 40-c...",$,POWER BOOST™ Ingredients Guaranteed For 12 Y...
1,Batteries,$20.99,No Discount,NaN,"Duracell Coppertop Alkaline AAA Batteries, 40-...",$,POWER BOOST™ Ingredients Guaranteed For 12 Y...
2,Batteries,$20.99,No Discount,NaN,"Duracell 9V Alkaline Batteries, 8-count",$,Long-Lasting Batteries Designed for Dependabil...
3,Batteries,$18.99,No Discount,NaN,"Duracell C Alkaline Batteries, 14-count",$,Long-Lasting Batteries Designed for Dependabil...
4,Batteries,$14.99,No Discount,Rated 4.8 out of 5 stars based on 125 reviews.,"Duracell 2032 Lithium Coin Batteries, 12-count",$,Our Longest Lasting Lithium Coin Battery Ever ...


In [33]:
items.isna().sum().sum()

14

In [34]:
items = items.dropna().reset_index(drop=True)

In [35]:
items.isna().sum().sum()

0

In [36]:
items.head()

,Sub Category,Price,Discount,Rating,Title,Currency,Feature
0,Batteries,$14.99,No Discount,Rated 4.8 out of 5 stars based on 125 reviews.,"Duracell 2032 Lithium Coin Batteries, 12-count",$,Our Longest Lasting Lithium Coin Battery Ever ...
1,Cameras & Camcorders,$289.99,Price valid through 1/21/24,Rated 4.2 out of 5 stars based on 238 reviews.,GoPro HERO11 Black Action Camera Bundle,$,"Also Includes: Shorty Hand Grip, Spare Enduro ..."
2,Cameras & Camcorders,$399.99,Price valid through 3/31/24,Rated 4.4 out of 5 stars based on 99 reviews.,Insta360 X3 - 5.7K 360° Action Camera Adventur...,$,X3 Captures 5.7K 360 Active HDR Video Dual 1...
3,Cameras & Camcorders,$429.99,No Discount,Rated 4.4 out of 5 stars based on 115 reviews.,DJI Mini 3 Drone Aerial Camera Bundle,$,Less Than 249g (0.55lbs) Ultralight and Foldab...
4,Cameras & Camcorders,$239.97,No Discount,Rated 4.1 out of 5 stars based on 311 reviews.,GoPro HERO10 Black Action Camera Bundle,$,"Also Includes: Extra Battery/Charger, Compact ..."


In [37]:
items2 = items[['Title', 'Feature', 'Price']].copy()

In [38]:
items2.rename(columns={'Title': 'ItemName', 'Feature': 'Description'}, inplace=True)

In [39]:
items2.head()

,ItemName,Description,Price
0,"Duracell 2032 Lithium Coin Batteries, 12-count",Our Longest Lasting Lithium Coin Battery Ever ...,$14.99
1,GoPro HERO11 Black Action Camera Bundle,"Also Includes: Shorty Hand Grip, Spare Enduro ...",$289.99
2,Insta360 X3 - 5.7K 360° Action Camera Adventur...,X3 Captures 5.7K 360 Active HDR Video Dual 1...,$399.99
3,DJI Mini 3 Drone Aerial Camera Bundle,Less Than 249g (0.55lbs) Ultralight and Foldab...,$429.99
4,GoPro HERO10 Black Action Camera Bundle,"Also Includes: Extra Battery/Charger, Compact ...",$239.97


In [40]:
items2['Price'] = items2['Price'].str.replace('$', '')
items2['Price'] = items2['Price'].str.replace(',', '')

C:\Users\quay\AppData\Local\Temp\ipykernel_19168\939474044.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  items2['Price'] = items2['Price'].str.replace('$', '')


In [41]:
items2['Price'] = pd.to_numeric(items2['Price'], errors='coerce')
items2 = items2.dropna(subset=['Price'])
items2['Price'] = items2['Price'].round(2)

In [42]:
items2.isna().sum().sum()

0

In [43]:
items2.shape

(621, 3)

In [44]:
items2['ItemID'] = range(1, len(items2) + 1)

In [45]:
items2.head()

,ItemName,Description,Price,ItemID
0,"Duracell 2032 Lithium Coin Batteries, 12-count",Our Longest Lasting Lithium Coin Battery Ever ...,14.99,1
1,GoPro HERO11 Black Action Camera Bundle,"Also Includes: Shorty Hand Grip, Spare Enduro ...",289.99,2
2,Insta360 X3 - 5.7K 360° Action Camera Adventur...,X3 Captures 5.7K 360 Active HDR Video Dual 1...,399.99,3
3,DJI Mini 3 Drone Aerial Camera Bundle,Less Than 249g (0.55lbs) Ultralight and Foldab...,429.99,4
4,GoPro HERO10 Black Action Camera Bundle,"Also Includes: Extra Battery/Charger, Compact ...",239.97,5


In [46]:
items2['UserID'] = np.random.randint(1, 2481, len(items2)) 

In [47]:
items2.head()

,ItemName,Description,Price,ItemID,UserID
0,"Duracell 2032 Lithium Coin Batteries, 12-count",Our Longest Lasting Lithium Coin Battery Ever ...,14.99,1,1548
1,GoPro HERO11 Black Action Camera Bundle,"Also Includes: Shorty Hand Grip, Spare Enduro ...",289.99,2,1190
2,Insta360 X3 - 5.7K 360° Action Camera Adventur...,X3 Captures 5.7K 360 Active HDR Video Dual 1...,399.99,3,1260
3,DJI Mini 3 Drone Aerial Camera Bundle,Less Than 249g (0.55lbs) Ultralight and Foldab...,429.99,4,232
4,GoPro HERO10 Black Action Camera Bundle,"Also Includes: Extra Battery/Charger, Compact ...",239.97,5,1736


In [48]:
items2 = items2[['ItemID', 'UserID', 'ItemName', 'Description', 'Price']]

In [49]:
items2['Description'] = items2['Description'].str[:200]

In [50]:
items2.head()

,ItemID,UserID,ItemName,Description,Price
0,1,1548,"Duracell 2032 Lithium Coin Batteries, 12-count",Our Longest Lasting Lithium Coin Battery Ever ...,14.99
1,2,1190,GoPro HERO11 Black Action Camera Bundle,"Also Includes: Shorty Hand Grip, Spare Enduro ...",289.99
2,3,1260,Insta360 X3 - 5.7K 360° Action Camera Adventur...,X3 Captures 5.7K 360 Active HDR Video Dual 1...,399.99
3,4,232,DJI Mini 3 Drone Aerial Camera Bundle,Less Than 249g (0.55lbs) Ultralight and Foldab...,429.99
4,5,1736,GoPro HERO10 Black Action Camera Bundle,"Also Includes: Extra Battery/Charger, Compact ...",239.97


In [51]:
items2.isna().sum().sum()

0

In [52]:
groupby = items2.groupby('UserID').size().reset_index(name='Count')
groupby[groupby['Count'] > 1].head()

,UserID,Count
12,60,2
21,93,2
54,226,2
61,267,2
66,281,2


### unit test


In [53]:
def test_price(df):
    for index, row in df.iterrows():
        price = row['Price']
        user_id = row['UserID']
        assert pd.api.types.is_numeric_dtype(type(price)), f"UserID {user_id}'s Price {price} is not numeric"
    print("All prices are numeric")
    
test_price(items2)

All prices are numeric


In [54]:
def is_valid_description(description: str) -> bool:
    return len(description) <= 200

def test_description_length(df):
    invalid_descriptions = df[~df['Description'].apply(is_valid_description)]
    assert invalid_descriptions.empty, f"Descriptions exceeding 200 characters found: {invalid_descriptions['Description'].tolist()}"
    print("All descriptions have 200 characters or less.")

test_description_length(items2)

All descriptions have 200 characters or less.


In [55]:
items2.shape

(621, 5)

## listings schema

In [56]:
num_entries = 50

# ListingID
listing_id = range(1, num_entries + 1)

# BorrowerID & LenderID
borrower_id = np.random.randint(1, 2481, size=num_entries)
lender_id = np.random.randint(1, 2481, size=num_entries)

for i in range(num_entries):
    while borrower_id[i] == lender_id[i]:
        lender_id[i] = np.random.randint(1, 2481)

# ItemID
item_ids = np.random.randint(1, 621, size=num_entries)

# StartDate & EndDate
start_date_base = datetime.strptime('01-01-2024', '%d-%m-%Y')
end_date_limit = datetime.strptime('01-09-2024', '%d-%m-%Y')

start_dates = [start_date_base + timedelta(days=random.randint(0, (end_date_limit - start_date_base).days)) for _ in range(num_entries)]
end_dates = [start_dates[i] + timedelta(days=random.randint(1, (end_date_limit - start_dates[i]).days)) for i in range(num_entries)]

# Creating Return_Flag column with all values as True (1)
return_flag = [1] * num_entries

# Create the DataFrame
listings = pd.DataFrame({
    'ListingID': listing_id,
    'BorrowerID': borrower_id,
    'LenderID': lender_id,
    'ItemID': item_ids,
    'StartDate': start_dates,
    'EndDate': end_dates,
    'ReturnFlag': return_flag
})



In [57]:
listings.isna().sum().sum()

0

In [58]:
listings.head()

,ListingID,BorrowerID,LenderID,ItemID,StartDate,EndDate,ReturnFlag
0,1,1098,1340,555,2024-04-17,2024-08-09,1
1,2,703,174,426,2024-06-16,2024-07-25,1
2,3,1855,533,188,2024-05-19,2024-06-10,1
3,4,1639,454,521,2024-03-08,2024-05-10,1
4,5,1497,1957,597,2024-01-08,2024-07-01,1


### small unit test practice

In [59]:
def test_borrower_lender_ids(df):
    for i in range(len(df)):
        assert df.loc[i, 'BorrowerID'] != df.loc[i, 'LenderID'], \
            f"BorrowerID and LenderID are the same for ListingID {df.loc[i, 'ListingID']}"
    print("No same ID")

test_borrower_lender_ids(listings)


No same ID


In [60]:
def test_invalid_dates(df):
    for i in range(len(df)):
        assert df.loc[i, 'EndDate'] > df.loc[i, 'StartDate'], \
            f"EndDate is not greater than StartDate for ListingID {df.loc[i, 'ListingID']}"
    print("No EndDate > StartDate")
test_invalid_dates(listings)

No EndDate > StartDate


# Export CSV

In [61]:
%pwd

'C:\\Users\\quay\\Documents\\ShareIt\\data\\raw'

In [62]:
cd ..

C:\Users\quay\Documents\ShareIt\data


In [63]:
cd clean/

C:\Users\quay\Documents\ShareIt\data\clean


In [64]:
users2.to_csv('users.csv', index=False)
items2.to_csv('items.csv', index=False) 
listings.to_csv('listings.csv', index=False) 